In [1]:
import subprocess

# Local imports
from input_writer import InputWriter
from output_reader import OutputReader
from filenames import (
    PHASE_SHIFT_INPUT_FILE,
    PHASE_SHIFT_OUTPUT_FILE,
    PHASE_SHIFT_EXECUTABLE,
)

In [2]:
!git checkout -- dphqq.d
!head -n5 dphqq.d

input for phases
jb,je       0  0
jborn       9
nj         32 
sing,tr,cp  1  0  0


Be sure to modify `jb,je` and `sing,tr,cp`

In [3]:
!head -n5 dphqq.d

input for phases
jb,je       0  4
jborn       9
nj         32 
sing,tr,cp  1  1  1


In [4]:
input_writer = InputWriter(PHASE_SHIFT_INPUT_FILE)

In [5]:
output_reader = OutputReader(PHASE_SHIFT_OUTPUT_FILE)

In [6]:
def run_phase_shift_executable():
    process = subprocess.run(["./" + PHASE_SHIFT_EXECUTABLE])
    assert process.returncode == 0

In [7]:
run_phase_shift_executable()

In [8]:
INITIAL_COEFFICIENTS = input_writer.coefficients
INITIAL_RESULTS = output_reader.get_results_from_output()

In [9]:
print(INITIAL_COEFFICIENTS)

{'1S0': (0.126, 0.3048), '1P1': (13.319, 0.849), '3S1': (-0.583795, 0.2399, 0.0, 0.0), '3PJ': (1.28, -0.52, -0.865, 0.0)}


In [10]:
print(INITIAL_RESULTS.keys())

dict_keys(['1S0', '3P0', '1P1', '3P1', '3S1', '3D1', 'E1', '1D2', '3D2', '3P2', '3F2', 'E2', '1F3', '3F3', '3D3', '3G3', 'E3', '1G4', '3G4', '3F4', '3H4', 'E4'])


In [13]:
PERTURBATION = 1.0

def determine_affected_states(state, coeff_index, threshold=0.0):
    input_writer.target_states = [state]
    initial_coefficients = list(INITIAL_COEFFICIENTS[state])
    
    new_coefficients = list(initial_coefficients)
    new_coefficients[coeff_index] += PERTURBATION
    
    # Modify input file and compute new output
    input_writer.modify_coefficients(new_coefficients)
    input_writer.write_lines()
    run_phase_shift_executable()
    results = output_reader.get_results_from_output()
    
    # Determine affected states
    affected_states = [
        state for state in INITIAL_RESULTS.keys()
        if results[state] - INITIAL_RESULTS[state] > threshold
    ]
    
    # Reset input file and output file
    input_writer.modify_coefficients(initial_coefficients)
    input_writer.write_lines()
    run_phase_shift_executable()
    
    return affected_states

In [14]:
determine_affected_states("1S0", 0)

['1S0', '1D2', '1G4']

In [15]:
state_dependencies = {
    (state, index): determine_affected_states(state, index)
    for state in input_writer.states
    for index in range(input_writer.multiplicity[state])
}

In [16]:
for k, v in state_dependencies.items():
    print(f"{k}: {v}")

('1S0', 0): ['1S0', '1D2', '1G4']
('1S0', 1): ['1S0', '1D2', '1G4']
('1P1', 0): ['1P1', '1F3']
('1P1', 1): ['1P1', '1F3']
('3S1', 0): ['3S1', '3D1', 'E1', '3D2', '3D3', '3G3', 'E3', '3G4']
('3S1', 1): ['3S1', '3D1', 'E1', '3D2', '3D3', '3G3', 'E3', '3G4']
('3S1', 2): ['3S1', '3D1', 'E1', '3D2', '3D3', '3G3', 'E3', '3G4']
('3S1', 3): ['3S1', '3D1', 'E1', '3D2', '3D3', '3G3', 'E3', '3G4']
('3PJ', 0): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F3', '3F4', '3H4', 'E4']
('3PJ', 1): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F3', '3F4', '3H4', 'E4']
('3PJ', 2): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F3', '3F4', '3H4', 'E4']
('3PJ', 3): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F3', '3F4', '3H4', 'E4']


In [22]:
THRESHOLD = 1.0

large_state_dependencies = {
    (state, index): determine_affected_states(
        state, index, threshold=THRESHOLD
    )
    for state in input_writer.states
    for index in range(input_writer.multiplicity[state])
}

In [23]:
for k, v in large_state_dependencies.items():
    print(f"{k}: {v}")

('1S0', 0): ['1S0', '1D2']
('1S0', 1): ['1S0', '1D2']
('1P1', 0): ['1P1']
('1P1', 1): ['1P1', '1F3']
('3S1', 0): ['3S1', '3D1', 'E1', '3D2', '3D3']
('3S1', 1): ['3S1', '3D1', 'E1', '3D2', '3D3']
('3S1', 2): ['3S1', '3D1', 'E1', '3D2', '3D3', '3G3', 'E3', '3G4']
('3S1', 3): ['3S1', '3D1', 'E1', '3D2', '3D3']
('3PJ', 0): ['3P0', '3P1', '3P2', 'E2']
('3PJ', 1): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F3', '3F4']
('3PJ', 2): ['3P0', '3P1', '3P2', '3F2', 'E2', '3F4']
('3PJ', 3): ['3P0', '3P1', '3P2', '3F2', '3F3', '3F4', 'E4']


In [24]:
!git checkout -- dphqq.d
!head -n5 dphqq.d

input for phases
jb,je       0  0
jborn       9
nj         32 
sing,tr,cp  1  0  0
